# Homework 5

Billy Nayden

## Import Packages

In [1]:
from nltk.metrics import *
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pprint

## 1. Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.
> a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.

> b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.

> c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.

> d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.

In [2]:
#https://www.dataquest.io/blog/web-scraping-tutorial-python/ used as a resource for this HW
from bs4 import BeautifulSoup
import requests

In [35]:
# Read in spreadsheet with links
df = pd.read_excel('C:\\Users\\WilliamNayden\\Documents\\MSDS\\20201019_IMDB_Reviews_for_NLP.xlsx')

link_list = df['Link'].tolist()

In [15]:
#Downloads the page
page = requests.get("https://www.imdb.com/review/rw5749269/?ref_=tt_urv")

In [16]:
#Raw content
page.content

b'\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///review/rw5749269?src=mdot">\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>arapacis-metal\'s Review of Star Wars: Episode IX - The Rise of Skywalker - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle",

## 2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:
> a.	In Python, use BeautifulSoup to grab the main review text from each link.

> b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser.

> c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.

In [17]:
#Structure the html so we can find the review text tag
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///review/rw5749269?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   arapacis-metal's Review of Star Wars: Episode IX - The Rise of Skywalker - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof u

In [18]:
list(soup.children)

['\n',
 'html',
 '\n',
 <html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="app-id=342792525, app-argument=imdb:///review/rw5749269?src=mdot" name="apple-itunes-app"/>
 <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
 <script>
     if (typeof uet == 'function') {
       uet("bb", "LoadTitle", {wb: 1});
     }
 </script>
 <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
 <title>arapacis-metal's Review of Star Wars: Episode IX - The Rise of Skywalker - IMDb</title>
 <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
 <script>
     if (typeof uet == 'function') {
       uet("be", "LoadTitle", {wb: 1});
     }
 </script>
 <script>
     if (typeof uex == 'function') {
  

In [19]:
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [20]:
html = list(soup.children)[3]

In [21]:
html

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///review/rw5749269?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>arapacis-metal's Review of Star Wars: Episode IX - The Rise of Skywalker - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
<

### Extract text from HTML document

In [22]:
span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()

In [23]:
span

"They are so used to making films for four-year olds that they thought the force is magic. There are magical and inexplicable wizardry happening all the time. Who knew 'Jedi' can heal the drying and mortally wounded? All those Jedi didn't need to die in episodes I-III! If only Yoda had known!!\nIt is not just that nurse Rey is so magical. People fly, people transport items across the galaxy, people come back from the dead and intervene/assist/interfere and the list continues.Twinkle twinkle little star...."

In [24]:
#NP chunker taken from: https://www.nltk.org/book/ch07.html

#Remove punctuation
punc= ".,`''\\"
import string

#Put everything in lowercase
span = span.lower()

#Remove punctuation for stemmer
for i in span:
    for j in punc:
        if i in punc:
            span = span.replace(i, "")
            
#First we need to tokenize and tag the sentence
tokenized = word_tokenize(span)
tagged = nltk.pos_tag(tokenized)

#Define that we want a determinant, aand a noun potentially preceeded by an adjective
np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

np_chunked = np.parse(tagged)

In [25]:
print(np_chunked)

(S
  they/PRP
  are/VBP
  so/RB
  used/JJ
  to/TO
  making/VBG
  (NP films/NNS)
  for/IN
  (NP four-year/JJ olds/NNS)
  that/IN
  they/PRP
  thought/VBD
  (NP the/DT force/NN)
  is/VBZ
  magic/JJ
  there/EX
  are/VBP
  magical/JJ
  and/CC
  (NP inexplicable/JJ wizardry/NN)
  happening/VBG
  all/PDT
  (NP the/DT time/NN)
  who/WP
  knew/VBD
  (NP jedi/NN)
  can/MD
  heal/VB
  (NP the/DT drying/NN)
  and/CC
  mortally/RB
  wounded/VBN
  ?/.
  all/DT
  (NP those/DT jedi/NNS)
  didnt/VBP
  need/VBP
  to/TO
  die/VB
  in/IN
  episodes/JJ
  i-iii/JJ
  !/.
  if/IN
  only/RB
  (NP yoda/NN)
  had/VBD
  known/VBN
  !/.
  !/.
  it/PRP
  is/VBZ
  not/RB
  just/RB
  that/IN
  (NP nurse/JJ rey/NN)
  is/VBZ
  so/RB
  (NP magical/JJ people/NNS)
  fly/VBP
  (NP people/NNS)
  transport/VBP
  (NP items/NNS)
  across/IN
  (NP the/DT galaxy/NN people/NNS)
  come/VBP
  back/RB
  from/IN
  the/DT
  dead/JJ
  and/CC
  intervene/assist/interfere/JJ
  and/CC
  (NP the/DT list/NN continuestwinkle/NN)
  (NP twink

## 3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).

In [36]:
output = {}
counter = 1

np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

for i in link_list:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    tokenized = word_tokenize(span)
    tagged = nltk.pos_tag(tokenized)
    chunked = np.parse(tagged)
    for subtree in chunked.subtrees():
        if subtree.label() == 'NP':
            lst.append(subtree)
    
    output["Review: " + str(counter)] = lst
    counter +=1

In [37]:
output

{'Review: 1': [Tree('NP', [('films', 'NNS')]),
  Tree('NP', [('four-year', 'JJ'), ('olds', 'NNS')]),
  Tree('NP', [('the', 'DT'), ('force', 'NN')]),
  Tree('NP', [('inexplicable', 'JJ'), ('wizardry', 'NN')]),
  Tree('NP', [('the', 'DT'), ('time', 'NN')]),
  Tree('NP', [("'Jedi", 'NNP')]),
  Tree('NP', [('the', 'DT'), ('drying', 'NN')]),
  Tree('NP', [('those', 'DT'), ('Jedi', 'NNP')]),
  Tree('NP', [('Yoda', 'NNP')]),
  Tree('NP', [('nurse', 'JJ'), ('Rey', 'NNP')]),
  Tree('NP', [('People', 'NNS')]),
  Tree('NP', [('people', 'NNS')]),
  Tree('NP', [('items', 'NNS')]),
  Tree('NP', [('the', 'DT'), ('galaxy', 'NN')]),
  Tree('NP', [('people', 'NNS')]),
  Tree('NP', [('the', 'DT'), ('list', 'NN'), ('continues.Twinkle', 'NN')]),
  Tree('NP', [('twinkle', 'JJ'), ('little', 'JJ'), ('star', 'NN')])],
 'Review: 2': [Tree('NP', [('Star', 'NNP'), ('Wars', 'NNP'), ('films', 'NNS')]),
  Tree('NP', [('this', 'DT'), ('film', 'NN')]),
  Tree('NP', [('Did', 'NNP'), ('anyone', 'NN')]),
  Tree('NP', [('

I selected Star Wars movies as my genre. I selected 12 reviews manually for each of the 9 Star Wars films, resulting in a data set of 108 reviews. I collected these from IMDB, put them in an Excel spreadsheet, and loaded that spreadsheet into the notebook for parsing.